In [29]:
import os
import sys
import numpy as np
from datetime import datetime

import copy
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import VOCDetection
import torchvision.transforms as transforms
from torchvision.models import alexnet, resnet50
from torchvision.transforms import functional as F
from xml.etree import ElementTree as ET
import pandas as pd
from PIL import Image

# models from torchvision
import torchvision.models as models
from torchvision.ops import RoIPool

sys.path.append('../../')  
from Object_Detection.RCNN import selective_search
from torchmetrics import Accuracy
from torchinfo import summary

from torch.utils.tensorboard import SummaryWriter


# Get Fast RCNN from torchvision


In [2]:
def parse_annotations(annotation):
    objects = annotation['annotation']['object']
    boxes = []
    classes = []
    for obj in objects:
        xmin = float(obj['bndbox']['xmin'])
        ymin = float(obj['bndbox']['ymin'])
        xmax = float(obj['bndbox']['xmax'])
        ymax = float(obj['bndbox']['ymax'])
        boxes.append([xmin, ymin, xmax, ymax])
        classes.append(obj['name'])
    return torch.tensor(boxes), classes  # Convert boxes to tensors, keep classes as list or map them to integers


In [18]:
# Define the transformation
transform = transforms.Compose([
    transforms.Resize((800, 800)),  # Resize images to a common size
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomVOCDataset(VOCDetection):
    def __getitem__(self, index):
        img, target = super(CustomVOCDataset, self).__getitem__(index)
        boxes, classes = parse_annotations(target)
        return img, boxes, classes

# Update the dataset instance with the custom class
dataset = CustomVOCDataset(root='./data/VOCdevkit/VOC2012', year='2012', image_set='train', download=True, transform=transform)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)


Using downloaded and verified file: ./data/VOCdevkit/VOC2012\VOCtrainval_11-May-2012.tar
Extracting ./data/VOCdevkit/VOC2012\VOCtrainval_11-May-2012.tar to ./data/VOCdevkit/VOC2012


In [28]:
gs = selective_search.get_selective_search()

counter, start_index = 0, 0

# Assuming 'voc_dataset' is iterable with (image, annot)
for image, boxes, classes in data_loader:
    
    counter += 1
    # Counter for images processed
    print(f'Processing image {counter}')

    print(image.shape)
    # Get proposals and annotations similar to your original code
    image_array = image.permute(0, 2, 3, 1).numpy()

    image_array = np.array(image_array)
    selective_search.config(gs, image_array, strategy='q')
    rects = selective_search.get_rects(gs)


    if counter == 1:
        break

Processing image 1
torch.Size([1, 3, 800, 800])


error: OpenCV(4.9.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0xcf03815f::Set<3,-1,-1>,struct cv::impl::A0xcf03815f::Set<0,5,-1>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [32]:
# Check selectivesearch

gs = selective_search.get_selective_search()


In [33]:
# image_array = np.array(img)
selective_search.config(gs, img_for_sc, strategy='q')


In [34]:
base_model = models.resnet50(weights=True)

# Remove the final fully connected layer (fc) to use as a feature extractor
modules = list(base_model.children())[:-2]  # Remove the last fully connected layer and avgpool
base_model = torch.nn.Sequential(*modules)


m:\ML\ML_regs\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [51]:
# Intermediate RoI pooling layer that takes selected region proposals and extracts features
# RoI pooling
roi_pool = RoIPool(output_size=(7, 7), spatial_scale=1/32)


In [36]:
rects = selective_search.get_rects(gs)

In [37]:
rects[0]

array([176, 566, 223, 585])

In [44]:
f = base_model(img.unsqueeze(0))

In [45]:
f.shape

torch.Size([1, 2048, 25, 25])

In [52]:
# pass the featire through the RoI pooling layer with the region proposals
roi_pool_feats = roi_pool(f, [torch.tensor([rects[0]]).float()])

In [ ]:
roi_pool_feats

In [15]:
num_classes = 21  # PASCAL VOC classes + background
classifier = torch.nn.Sequential(
    torch.nn.Linear(512 * 7 * 7, 4096),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(4096, 4096),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(4096, num_classes + 4)  # class scores and bbox regressor outputs
)

